In [2]:
from sqlalchemy import MetaData, select, text, null, literal_column, union_all
from sqlalchemy import types as sqltypes, func as sqlfunc

In [3]:
import sys  
sys.path.insert(0, 'tests')

from test_rdb2rdf import setup_engine, create_database

In [55]:
from pathlib import Path
from sqlalchemy import create_engine, engine as sqlengine
db = setup_engine("sqlite", echo=False)
# db = setup_engine("duckdb", echo=True)

# path = Path('tests/rdb2rdf-ts/D008-1table1compositeprimarykey3columns1row')
# path = Path('tests/rdb2rdf-ts/D009-2tables1primarykey1foreignkey')
# path = Path('tests/rdb2rdf-ts/D000-1table1column0rows')
path = Path('tests/rdb2rdf-ts/D001-1table1column1row')


create_database(db, path, 'create.sql')
print(db.dialect.server_version_info)


metadata = MetaData()
metadata.reflect(db)
metadata.tables

(3, 31, 1)


FacadeDict({'Student': Table('Student', MetaData(), Column('Name', VARCHAR(length=50), table=<Student>), schema=None)})

In [101]:
import rdflib
from rdflib_r2r import R2RStore, R2RMapping
from rdflib_r2r.r2r_store import ColForm, sql_pretty
mapping = R2RMapping.from_db(db) # direct mapping
store = R2RStore(db=db, mapping=mapping)

pattern = (None, None, None)
query, subforms = store.queryPattern(metadata, pattern)

# print(sql_pretty(query))
# with db.connect() as conn:
#     display(list(conn.execute(query)))

query = query.subquery()
cols = getattr(query, "exported_columns", query.c)
display(list(cols))
onlycols = []
for subform, colname in zip(subforms, "spog"):
    colform = ColForm.from_subform(cols, *subform)
    onlycols.append(colform.expr().label(colname))

query = select(*onlycols)

with db.connect() as conn:
    print(list(conn.execute(query)))


@base <http://example.com/base/> .
@prefix ns1: <http://www.w3.org/ns/r2rml#> .

[] a ns1:TriplesMap ;
    ns1:logicalTable [ ns1:tableName "\"Student\"" ] ;
    ns1:predicateObjectMap [ ns1:objectMap [ ns1:column "\"Name\"" ] ;
            ns1:predicate <Student#Name> ] ;
    ns1:subjectMap [ ns1:class <Student> ;
            ns1:termType ns1:BlankNode ] .




[<sqlalchemy.sql.elements.ColumnClause at 0x7fe9cb301e50; s>,
 <sqlalchemy.sql.elements.ColumnClause at 0x7fe9cb3016d0; p>,
 <sqlalchemy.sql.elements.ColumnClause at 0x7fe9cb3018e0; o>,
 <sqlalchemy.sql.elements.ColumnClause at 0x7fe9cb3019d0; g>]

[('_:Student#1', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://example.com/base/Student>', None), ('_:Student#1', '<http://example.com/base/Student#Name>', 'Venus', None)]


In [86]:
with db.connect() as conn:
#     result = conn.execute(text("""SELECT rowid, Name FROM Student"""))
    result = conn.execute(text("""
    SELECT CAST(:param_1 AS VARCHAR) || CAST(CAST("Student".rowid AS VARCHAR) AS VARCHAR) AS s, 
           '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>' AS p, 
           '<http://example.com/base/Student>' AS o, 
           NULL AS g
    FROM "Student"
    """), {"param_1":"_:Student#"})
    display(result.all())
    
    result = conn.execute(text("""
    SELECT CAST('_:Student#' AS VARCHAR) || CAST(CAST("Student".rowid AS VARCHAR) AS VARCHAR) AS s,
       '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>' AS p,
       '<http://example.com/base/Student>' AS o,
       NULL AS g
    FROM "Student"
    """))
    display(result.all())

[('_:Student#1', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://example.com/base/Student>', None)]

[('_:Student#1', '<http://www.w3.org/1999/02/22-rdf-syntax-ns#type>', '<http://example.com/base/Student>', None)]

# DuckDB stuff

In [101]:
import duckdb
conn = duckdb.connect(database=':memory:', read_only=False)
conn.execute("""
CREATE TABLE "Student" (
    "ID" integer,
    "Name" varchar(50),
    "Sport" varchar (50),
    PRIMARY KEY ("ID","Name")
);
INSERT INTO "Student" ("ID", "Name","Sport") VALUES(10,'Venus Williams','Tennis');
""")
print(conn.execute("PRAGMA table_info('Student');", (), None).fetchall())
print(conn.execute("select * from duckdb_indexes", (), None).fetchall())
print(conn.execute("""
SELECT
                cons.conname as name,
                cons.conkey as key,
                a.attnum as col_num,
                a.attname as col_name
            FROM
                pg_catalog.pg_constraint cons
                join pg_attribute a
                  on cons.conrelid = a.attrelid
""", (), None).fetchall())

[(0, 'ID', 'INTEGER', True, None, True), (1, 'Name', 'VARCHAR', True, None, True), (2, 'Sport', 'VARCHAR', False, None, False)]
[]
[('PRIMARY KEY("ID", "Name")', [1, 0], 3, 'Sport'), ('NOT NULL', [0], 3, 'Sport'), ('NOT NULL', [1], 3, 'Sport'), ('PRIMARY KEY("ID", "Name")', [1, 0], 2, 'Name'), ('NOT NULL', [0], 2, 'Name'), ('NOT NULL', [1], 2, 'Name'), ('PRIMARY KEY("ID", "Name")', [1, 0], 1, 'ID'), ('NOT NULL', [0], 1, 'ID'), ('NOT NULL', [1], 1, 'ID')]
